In [ ]:
# ----------------------------- Tick data count analysis for market feed "Cell: 1" -----------------------------

import os
import pandas as pd
import struct
from pathlib import Path
from datetime import datetime
from dhanhq import MarketFeed
import ast
import sys; sys.path.append('..'); from config import MYPATHS, XDATA
import sys; sys.path.append('..'); import functions as f

class TickFileReader:
    """Fast reader for tick files - supports parallel reading"""
    
    RECORD_SIZE = 32

    @staticmethod
    def read_full_file(filepath):
        """Read entire file - returns all ticks"""
        ticks = []
        try:
            with open(filepath, 'rb') as f_file:
                while True:
                    data = f_file.read(TickFileReader.RECORD_SIZE)
                    if len(data) != TickFileReader.RECORD_SIZE:
                        break
                    
                    time_bytes, price, oi, volume = struct.unpack('8sddd', data)
                    ltt = time_bytes.rstrip(b'\x00').decode('utf-8')
                    
                    if ltt:
                        ticks.append({
                            'LTT': ltt,
                            'price': price,
                            'OI': oi,
                            'volume': volume
                        })
        except Exception as e:
            print(f"❌ Error reading full file: {e}")
        
        return ticks

def read_all_ticks(folder_path, file_name):
    """Read all ticks for a security"""
    return TickFileReader.read_full_file(Path(folder_path) / f"{file_name}.bin")


In [ ]:
# ----------------------------- Tick data count analysis for market feed "Cell: 2" -----------------------------

market_feed_data = [{'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62407, 'LTP': '115.25', 'LTT': '09:15:58'},
    {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62371, 'LTP': '852.55', 'LTT': '15:29:21'},
    {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62375, 'LTP': '788.10', 'LTT': '09:15:48'},
    {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62414, 'LTP': '57.65', 'LTT': '09:15:57'},
    {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62442, 'LTP': '653.15', 'LTT': '15:29:38'},
    {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62409, 'LTP': '78.70', 'LTT': '09:15:58'},
    {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62410, 'LTP': '35.90', 'LTT': '09:15:58'},
    {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62387, 'LTP': '494.60', 'LTT': '09:15:57'},
    {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62373, 'LTP': '798.30', 'LTT': '15:25:17'},
    {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62421, 'LTP': '8.75', 'LTT': '09:15:58'},
    {'type': 'Ticker Data', 'exchange_segment': 2, 'security_id': 62399, 'LTP': '251.90', 'LTT': '09:15:58'}]


In [ ]:
# ----------------------------- Tick data count analysis for market feed "Cell: 3" -----------------------------

data_path=MYPATHS['base']
date = XDATA['Analysis_date']

file_path = MYPATHS['Option_Chain_with_Security_id'] + f"//{date}.csv"
option_chain_df = pd.read_csv(file_path)

filtered_df = f.get_prices_under_1000(option_chain_df)
instruments = f.create_instruments_from_df(filtered_df)

# -----------------------------------------------

folder_name = "Simulation_Tick_Data"        # "Simulation_Tick_Data" "Live_Tick_Data"
from_where_to_read = "from_file"            # "raw_output" "from_folders_file" "from_file"

# -----------------------------

folder_path = Path(data_path) / folder_name / f"market_data_{date}"

if from_where_to_read == "raw_output":
    
    market_feed_data_df = pd.DataFrame(market_feed_data)

    security_id_counts = market_feed_data_df['security_id'].value_counts()

if from_where_to_read == "from_folders_file":
    print(f"Single file : {folder_path}")

    all_ticks = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".bin"):
            file_name = file_name.split(".")[0]
            ticks = read_all_ticks(folder_path, file_name)
            all_ticks.extend([int(file_name)] * int(len(ticks)))

    all_ticks_df = pd.DataFrame(all_ticks, columns=["security_id"])
    security_id_counts = all_ticks_df['security_id'].value_counts()

if from_where_to_read == "from_file":

    market_feed_data = []
    file_path = MYPATHS['Raw_data_txt'] + f"\\{date}.txt"
    print(f"Single file : {file_path}")

    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():
                market_feed_data.append(ast.literal_eval(line))

    market_feed_data_df = pd.DataFrame(market_feed_data)

    security_id_counts = market_feed_data_df['security_id'].value_counts()

# -----------------------------------------------

filtered_df["counts_ce"] = filtered_df["security_id_CE"].map(security_id_counts).fillna(0).astype(int)
filtered_df["counts_pe"] = filtered_df["security_id_PE"].map(security_id_counts).fillna(0).astype(int)

cols = filtered_df.columns.tolist()

cols.insert(0, cols.pop(cols.index("counts_ce")))
cols.append(cols.pop(cols.index("counts_pe")))

filtered_df = filtered_df[cols]

total_count = filtered_df["counts_ce"].sum() + filtered_df["counts_pe"].sum()

print(f"Total instruments : {len(instruments)}, Total tick count (Simulated bin file) : {total_count}")
filtered_df